In [2]:
import mysql.connector
import re
dataset_name='lccgame'

def querymysql(query_command):
    db=mysql.connector.connect(host='localhost',
                           database='lccgame',
                           user='host',
                           password='host')
    cursor=db.cursor()
    cursor.execute(query_command)
    db.close()
    queryresult=cursor.fetchall()
    return queryresult

querycommand_usermsgs="""SELECT gamemsgs.gameid,playerinfo.interid,playerinfo.userid,playerinfo.playerrole,gamemsgs.msgbody 
                            FROM playerinfo 
                            INNER JOIN gamemsgs 
                            ON playerinfo.interid=gamemsgs.interid"""
interdata_msgs=querymysql(querycommand_usermsgs)



In [9]:
userid=14


responderaccept_ratio=[]
responderreject_ratio=[]
role='responder'+'%'
repattern_gamemsg = re.compile(r'e\((?P<msgname>\w*)\((?P<msgvars>.*)\)\)')
repattern_userinfo = re.compile(r'(?P<userrolename>\w*)\((?P<agentrolevars>.*)\)')
#get the choice that the responder makes
querycommand_rc="""SELECT gamemsgs.msgbody,playerinfo.playerrole,playerinfo.interid
                        FROM playerinfo
                        INNER JOIN gamemsgs 
                        ON playerinfo.interid=gamemsgs.interid
                        WHERE ( playerinfo.userid=%s 
                            AND playerinfo.playerrole LIKE "%s"
                            AND playerinfo.playerrole=gamemsgs.msgsenderrole )
                        ORDER BY playerinfo.interid
                    """%(userid,role)
msglist_responder_send=querymysql(querycommand_rc)#a(acceptornot(accept#3))
# extract offer values from the messages
for i in msglist_responder_send:
    responder_msg=re.search(repattern_gamemsg,i[0]).group('msgvars').split('#')
    responder_choice=responder_msg[0]
    proposer_offer=responder_msg[1]
    game_total=re.search(repattern_userinfo,i[1]).group('agentrolevars')
    if responder_choice=="reject":
        respondeechoice_ratio=responderreject_ratio+[int(proposer_offer)/int(game_total)]
    elif responder_choice=="accept":
        responderaccept_ratio=responderaccept_ratio+[int(proposer_offer)/int(game_total)]
    else:
        print("wired choice!")
    
print(responderreject_ratio)   
print(responderaccept_ratio)
print(len(msglist_responder_send))
    #responderget=responderget+[[int(investor_offer[0])/int(game_total[0]),i[2]]]




[0.3, 0.5, 0.2, 0.3, 0.3, 0.4, 0.4, 0.1, 0.3]
[0.5, 0.7]
11
